In [1]:
# === 0) Librerías ===
import pandas as pd
from pathlib import Path

In [6]:
# Ruta ABSOLUTA a la carpeta base del proyecto en WSL
BASE = Path("/home/luisrueda/PROYECTO-IA")
# Construimos la ruta al CSV uniendo carpetas/archivo de forma segura
RUTA = BASE / "data" / "IoT_AquaSensors_Crudo.csv"
# Mostramos la ruta completa del archivo que intentaremos leer
print("Archivo que voy a leer:", RUTA)
# Leemos el CSV como texto (dtype=str) para tener control total de tipos; 
# low_memory=False evita inferencias parciales de tipos en archivos grandes
df = pd.read_csv(RUTA, dtype=str, low_memory=False)
# Confirmamos dimensiones del DataFrame (filas, columnas)
print("OK, cargado:", df.shape)

Archivo que voy a leer: /home/luisrueda/PROYECTO-IA/data/IoT_AquaSensors_Crudo.csv
OK, cargado: (74796, 10)


In [ ]:
# === Paso 2: Crear marca de tiempo (Date + Time) ===
# Unir las columnas de fecha y hora en una sola
#elimina las filas invalidas
df["marca_tiempo"] = pd.to_datetime(
    df["Date"].astype(str).str.strip() + " " + df["Time"].astype(str).str.strip(),
    dayfirst=True,  # formato día-mes-año
    errors="coerce" # si no puede convertir, deja NaT
)
# Contar cuántas filas no se pudieron convertir
filas_invalidas = df["marca_tiempo"].isna().sum()
print("Filas eliminadas por fecha/hora inválida:", filas_invalidas)
# Eliminar las filas inválidas
df = df.dropna(subset=["marca_tiempo"])

Filas eliminadas por fecha/hora inválida: 51


In [ ]:
# === Paso 3: Normalizar estación (a 'estacion') ===
#Esto deja la columna estacion lista y consistente (sin espacios, minúsculas, nombres unificados).
# 1) Crear columna 'estacion' en español, limpia y en minúsculas
df["estacion"] = (df["station"].astype(str)
                  .str.strip()           # quitar espacios extremos
                  .str.lower()           # a minúsculas
                  .str.replace(r"\s+", "", regex=True))  # quitar espacios internos

# 2) Unificar variantes a un nombre canónico en español
mapa_estaciones = {
    "station1": "estacion1",
    "station2": "estacion2",
    "station3": "estacion3"
}
df["estacion"] = df["estacion"].replace(mapa_estaciones)
# 3) eliminar la columna original en inglés
df = df.drop(columns=["station"])

# 4) Chequeos rápidos
print("Conteo por estación:\n", df["estacion"].value_counts())
print("Nulos en 'estacion':", df["estacion"].isna().sum())


Conteo por estación:
 estacion
estacion3    25482
estacion2    25481
estacion1    23782
Name: count, dtype: int64
Nulos en 'estacion': 0


In [17]:
# === Paso 4: Convertir columnas de sensores a numérico ===

# 1) Definir nombres en español (mapea desde tus columnas originales)
mapa_columnas = {
    "NITRATE(PPM)": "nitrato_ppm",
    "PH": "ph",
    "AMMONIA(mg/l)": "amoniaco_mg_l",
    "TEMP": "temperatura_c",
    "DO": "oxigeno_disuelto_mg_l",
    "TURBIDITY": "turbidez_ntu",
    "MANGANESE(mg/l)": "manganeso_mg_l"
}
# 2) Renombrar columnas
df = df.rename(columns=mapa_columnas)
# 3) Forzar a numérico (reemplaza coma por punto y limpia espacios)
columnas_numericas = list(mapa_columnas.values())
for col in columnas_numericas:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace(",", ".", regex=False)
        .str.strip()
    )
    df[col] = pd.to_numeric(df[col], errors="coerce")
    # 4) Chequeo rápido de NaN por columna numérica
print("NaN por columna numérica:")
print(df[columnas_numericas].isna().sum().sort_values(ascending=False))
# 5) Vista rápida
print("\nVista rápida tras conversión:")
print(df[["marca_tiempo","estacion"] + columnas_numericas].head(5))


NaN por columna numérica:
manganeso_mg_l           25
oxigeno_disuelto_mg_l    10
nitrato_ppm               6
temperatura_c             6
ph                        2
amoniaco_mg_l             0
turbidez_ntu              0
dtype: int64

Vista rápida tras conversión:
         marca_tiempo   estacion  nitrato_ppm   ph  amoniaco_mg_l  \
0 2022-02-01 08:00:00  estacion1         18.3  5.7          0.010   
1 2022-02-01 08:20:00  estacion1          3.6  5.1          0.094   
2 2022-02-01 08:40:00  estacion1         13.1  5.5          0.060   
3 2022-02-01 09:00:00  estacion1         18.1  5.2          0.018   
4 2022-02-01 09:20:00  estacion1         10.8  5.2          0.038   

   temperatura_c  oxigeno_disuelto_mg_l  turbidez_ntu  manganeso_mg_l  
0          23.20                   11.6          31.7            0.71  
1          23.41                   10.5          18.8            0.62  
2          23.63                   10.3          23.2            0.73  
3          23.64               

In [21]:
# === Paso 5: Eliminar duplicados ===

# Guardamos número de filas antes
filas_antes = len(df)

# Ordenamos y quitamos duplicados
df = (
    df.sort_values(["estacion", "marca_tiempo"])
      .drop_duplicates(subset=["estacion", "marca_tiempo"], keep="first")
)
# Contamos cuántos se eliminaron
filas_despues = len(df)
duplicados_eliminados = filas_antes - filas_despues
print("Duplicados eliminados:", duplicados_eliminados)
print("Tamaño final del dataset:", df.shape)

Duplicados eliminados: 0
Tamaño final del dataset: (74745, 11)


In [ ]:
# === Paso 6: Manejo de valores faltantes (NaN) ===
#Reemplazar los huecos con la mediana por estación (más realista que usar una sola media global).
# 1) Lista de columnas de sensores
sensores = [
    "nitrato_ppm", "ph", "amoniaco_mg_l",
    "temperatura_c", "oxigeno_disuelto_mg_l",
    "turbidez_ntu", "manganeso_mg_l"
]
# 2) Revisar cuántos NaN hay por sensor
print("Valores faltantes (NaN) por columna:")
print(df[sensores].isna().sum())
# 3) Imputar valores faltantes con la MEDIANA de cada estación
#    Esto asegura que cada estación use sus propios valores típicos
for col in sensores:
    df[col] = df.groupby("estacion")[col].transform(
        lambda serie: serie.fillna(serie.median())
    )
# 4) Verificar nuevamente si quedaron NaN
print("\nValores faltantes tras imputación:")
print(df[sensores].isna().sum())

Valores faltantes (NaN) por columna:
nitrato_ppm               6
ph                        2
amoniaco_mg_l             0
temperatura_c             6
oxigeno_disuelto_mg_l    10
turbidez_ntu              0
manganeso_mg_l           25
dtype: int64

Valores faltantes tras imputación:
nitrato_ppm              0
ph                       0
amoniaco_mg_l            0
temperatura_c            0
oxigeno_disuelto_mg_l    0
turbidez_ntu             0
manganeso_mg_l           0
dtype: int64
